In [4]:
from unittest import result
import cv2
from matplotlib.backend_tools import SaveFigureBase
import mediapipe as mp
from numpy import angle, save
import math
import numpy as np
import csv

csvname = input('輸入動作編號 (使用數字)：')
csvdata="pose04.csv"

cap=cv2.VideoCapture(0)
mpHands = mp.solutions.hands #手部追蹤模組
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils
num=[1,5,9,13,17]

def cos1(ax,ay,bx,by,cx,cy):
    #handangle1=((cx-bx)*(ax-bx)+(cy-by)*(ay-by))/(((cx-bx)**2)**0.5+((cy-by)**2)**0.5)
    handangle1=((ax-bx)*(cx-bx)+(ay-by)*(cy-by))/((((ax-bx)**2+(ay-by)**2)**0.5)*(((cx-bx)**2+(cy-by)**2)**0.5))
    handangle2=np.degrees(np.arccos(handangle1))
    return handangle2
def howlong(ax,ay,bx,by):
    fingerlong=round(abs(((ax-bx)**2+(ay-by)**2)**0.5),2)
    return fingerlong
while True:
    ret,img=cap.read() # 布林值變數及影像變數
    file=open(csvdata,"a", newline="")
    imgH=img.shape[0]
    imgW=img.shape[1]
    if ret==True:
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #把圖片格式轉換成mediapipe想要的
        result=hands.process(imgRGB)
        # print(result.multi_hand_landmarks)
        if result.multi_hand_landmarks:
            for handlms in result.multi_hand_landmarks:#handlms=每一隻手
                mpDraw.draw_landmarks(img,handlms,mpHands.HAND_CONNECTIONS)
                hand=[]
                handangle=[]
                handlong=[]
                allcsvdata=[]
                long0_5=0
                long0_17=0
                long5_17=0
                Proportion0_5=[]
                Proportion0_17=[]
                Proportion5_17=[]
                for i,lm in enumerate(handlms.landmark):
                    Xpoint=lm.x
                    Ypoint=lm.y*imgH,2
                    #print(i,Xpoint,Ypoint)
                    hand.append((Xpoint,Ypoint))
                    #print(hand)                     
                #for i in range(0,21):
                #    print(hand[i][0],hand[i][1])
                for i in num:
                    saveangle=cos1(hand[0][0],hand[0][1],hand[i][0],hand[i][1],hand[i+1][0],hand[i+1][1])
                    handangle.append(round(saveangle,2))
                    saveangle=cos1(hand[i][0],hand[i][1],hand[i+1][0],hand[i+1][1],hand[i+2][0],hand[i+2][1])
                    handangle.append(round(saveangle,2))
                    saveangle=cos1(hand[i+1][0],hand[i+1][1],hand[i+2][0],hand[i+2][1],hand[i+3][0],hand[i+3][1])
                    handangle.append(round(saveangle,2))
                    fingerlong=howlong(hand[0][0],hand[i][0],hand[0][1],hand[i][1])
                    handlong.append(fingerlong)
                    fingerlong=howlong(hand[i][0],hand[i+1][0],hand[i][1],hand[i+1][1])
                    handlong.append(fingerlong)
                    fingerlong=howlong(hand[i+1][0],hand[i+2][0],hand[i+1][1],hand[i+2][1])
                    handlong.append(fingerlong)
                    fingerlong=howlong(hand[i+2][0],hand[i+3][0],hand[i+2][1],hand[i+3][1])
                    handlong.append(fingerlong)
                long0_17=fingerlong=howlong(hand[0][0],hand[17][0],hand[0][1],hand[17][1])
                long0_5=fingerlong=howlong(hand[0][0],hand[5][0],hand[0][1],hand[5][1])
                long5_17=fingerlong=howlong(hand[5][0],hand[17][0],hand[5][1],hand[17][1])
                for i in range(0,20):
                    savee=round(handlong[i]/long0_5,5)
                    Proportion0_5.append(savee)
                    savee=round(handlong[i]/long0_17,5)
                    Proportion0_17.append(savee)
                    savee=round(handlong[i]/long5_17,5)
                    Proportion5_17.append(savee)
                allcsvdata.append(csvname)
                allcsvdata.extend(handangle)
                allcsvdata.extend(handlong)
                allcsvdata.extend(Proportion0_5)
                allcsvdata.extend(Proportion0_17)
                allcsvdata.extend(Proportion5_17)
                #print(allcsvdata)
                writer = csv.writer(file)
                writer.writerow(allcsvdata)
                
                #print(handangle)
        
        cv2.imshow('img',img)# 顯示每一偵

    if cv2.waitKey(1) ==ord('q'):
        file.close()
        break

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

# xy版

In [3]:
import cv2
import mediapipe as mp
import csv

csvname = input('輸入動作編號 (使用數字)：')
csvdata = "pose10.csv"

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

while True:
    ret, img = cap.read()
    file = open(csvdata, "a", newline="")
    imgH = img.shape[0]
    imgW = img.shape[1]
    if ret == True:
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(imgRGB)
        if result.multi_hand_landmarks:
            for handlms in result.multi_hand_landmarks:
                mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                allcsvdata = [csvname]  # 初始化allcsvdata为包含csvname的列表
                for i, lm in enumerate(handlms.landmark):
                    allcsvdata.extend([lm.x, lm.y])  # 将Xpoint和Ypoint追加到allcsvdata列表中
                with open(csvdata, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(allcsvdata)

        cv2.imshow('img', img)

    if cv2.waitKey(1) == ord('q'):
        file.close()
        break

cap.release()
cv2.destroyAllWindows()
